### Load libs

In [1]:
import pandas as pd
import json
import numpy as np
from pandarallel import pandarallel
pandarallel.initialize()


INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
n = 500
pd.set_option('display.max_columns', n)
pd.set_option('display.max_rows', n)
pd.set_option('display.max_colwidth', -1)

/anaconda3/envs/kn/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


### Load data

In [3]:
term = 'AmeriCredit Automobile Receivables Trust 2017-1 Data Tape'


In [4]:
# load abs
folder = 'data/combined/'
file = '{}.csv'.format(term)
path = folder + file
data = pd.read_csv(path)
data.shape


/anaconda3/envs/kn/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (38,39,57) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1511394, 73)

In [5]:
# load mapper
m_folder = 'data/dictionary/mapper/'
m_file = 'mapper.json'
m_path = m_folder + m_file
with open(m_path) as f:
    mapper = json.load(f)
m_cols = list(mapper.keys())
    

### Formatting

In [6]:
def convert_id(row, column):
    
    """
    Convert ids
    """
    
    init = str(row[column])
    
    cleaned = init.replace('=', '').replace('"', '').strip()
    
    return cleaned
    

In [7]:
def reorder_date(row, column):
    
    """
    Reorder date
    """
    
    init = str(row[column])
    if init != '-':
        if '/' not in init:
            y = init[6:10]
            m = init[0:2]
            d = init[3:5]
            date = y + '-' + m + '-' + d
        elif '/' in init:
            y = init[3:7]
            m = init[0:2]
            date = y + '-' + m
    else:
        date = ''
    
    return date
        

In [8]:
data['ID'] = data.parallel_apply(convert_id, args = ('assetNumber', ), axis = 1)


In [9]:
# date cols
date_cols = ['reportingPeriodBeginningDate', 'reportingPeriodEndingDate', 'originationDate', 'loanMaturityDate', 'originalFirstPaymentDate', 'interestPaidThroughDate', 'zeroBalanceEffectiveDate', 'mostRecentServicingTransferReceivedDate', 'DemandResolutionDate'] 


In [10]:
for col in date_cols:
    print(col)
    data['{}R'.format(col)] = data.parallel_apply(reorder_date, args = (col, ), axis = 1)
    

reportingPeriodBeginningDate
reportingPeriodEndingDate
originationDate
loanMaturityDate
originalFirstPaymentDate
interestPaidThroughDate
zeroBalanceEffectiveDate
mostRecentServicingTransferReceivedDate
DemandResolutionDate


In [11]:
replacer_cols = ['originalLoanAmount', 'originalLoanTerm', 'originalInterestRatePercentage', 'gracePeriodNumber', 'obligorCreditScore', 'paymentToIncomePercentage', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue', 'reportingPeriodInterestRatePercentage', 'nextInterestRatePercentage', 'servicingFeePercentage', 'servicingFlatFeeAmount', 'otherServicerFeeRetainedByServicer', 'otherAssessedUncollectedServicerFeeAmount', 'scheduledInterestAmount', 'scheduledPrincipalAmount', 'otherPrincipalAdjustmentAmount', 'reportingPeriodActualEndBalanceAmount', 'reportingPeriodScheduledPaymentAmount', 'totalActualAmountPaid', 'actualInterestCollectedAmount', 'actualPrincipalCollectedAmount', 'actualOtherCollectedAmount', 'servicerAdvancedAmount', 'currentDelinquencyStatus', 'repurchaseAmount', 'chargedoffPrincipalAmount', 'recoveredAmount', 'repossessedProceedsAmount']    


In [12]:
data[replacer_cols] = data[replacer_cols].replace('-', np.nan)

In [13]:
# clean cols
clean_cols = ['currentDelinquencyStatus']
for col in clean_cols:
    data[col] = data[col].str.strip()
    data[col] = data[col].astype(float)
    

In [14]:
def replace_val(row, column):
    
    """
    Replace numeric values
    """
    
    init = str(row[column]).strip().replace(';', '')
    if init in ['0', '1', '2', '3', '4', '5', '98', '99']:
        mapped = mapper[column][init]
        return mapped
    else:
        if init[0] in ['0', '1', '2', '3', '4', '5']:
            use = init[0]
        elif init == '-':
            use_keys = list(mapper[column].keys())
            if '98' in use_keys:
                use = '98'
            elif '99' in use_keys:
                use = '99'
        else:
            use = init
        mapped = mapper[column][use]
    return mapped
    

In [15]:
for col in m_cols:
    print(col)
    new_col = col + 'M'
    data[new_col] = data.parallel_apply(replace_val, args = (col, ), axis = 1)
    

interestCalculationTypeCode
originalInterestRateTypeCode
paymentTypeCode
subvented
vehicleNewUsedCode
vehicleTypeCode
vehicleValueSourceCode
obligorIncomeVerificationLevelCode
obligorEmploymentVerificationCode
servicingAdvanceMethodCode
zeroBalanceCode
assetSubjectDemandStatusCode
repurchaseReplacementReasonCode
modificationTypeCode


In [16]:
def acct_status(row, b_col, e_col, zero_col, thresh):
    
    """
    Create karus account status
    """
    
    b = float(row[b_col])
    e = float(row[e_col])
    z = str(row[zero_col])
    
    if z in ['Charged-off', 'Repurchased or Replaced']:
        res = z
        return res
    if b < thresh and e < thresh:
        res = 'Prepaid or Matured'
        return res
    if z in ['Unavailable', 'Prepaid or Matured']:
        res = z
        return res
    

In [17]:
b_col = 'reportingPeriodBeginningLoanBalanceAmount'
e_col = 'nextReportingPeriodPaymentAmountDue'
z_col = 'zeroBalanceCodeM'
thresh = 50


In [18]:
data['accountStatus'] = data.parallel_apply(acct_status, args = (b_col, e_col, z_col, thresh, ), axis = 1)

In [19]:
data['accountStatus'].value_counts()

Unavailable                1458409
Charged-off                26714  
Prepaid or Matured         26234  
Repurchased or Replaced    37     
Name: accountStatus, dtype: int64

### Create static file

In [41]:
iloc_cols = list(set(['originatorName', 'primaryLoanServicerName', 'originationDateR', 'originalLoanAmount', 'originalLoanTerm', 'loanMaturityDateR', 'originalInterestRatePercentage', 'interestCalculationTypeCodeM', 'originalInterestRateTypeCodeM', 'originalInterestOnlyTermNumber', 'originalFirstPaymentDateR', 'underwritingIndicator', 'paymentTypeCodeM', 'vehicleManufacturerName', 'vehicleModelName', 'vehicleNewUsedCodeM', 'vehicleModelYear', 'vehicleTypeCodeM', 'vehicleValueAmount', 'vehicleValueSourceCodeM', 'obligorCreditScoreType', 'obligorIncomeVerificationLevelCodeM', 'obligorEmploymentVerificationCodeM', 'coObligorIndicator', 'paymentToIncomePercentage', 'obligorGeographicLocation', 'assetAddedIndicator', 'reportingPeriodModificationIndicator', 'servicingAdvanceMethodCodeM', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue', 'reportingPeriodInterestRatePercentage', 'nextInterestRatePercentage', 'scheduledInterestAmount', 'otherPrincipalAdjustmentAmount', 'reportingPeriodActualEndBalanceAmount', 'reportingPeriodScheduledPaymentAmount', 'assetSubjectDemandIndicator', 'zeroBalanceEffectiveDateR']))         
min_max_cols = list(set(['reportingPeriodBeginningDateR', 'reportingPeriodEndingDateR', 'remainingTermToMaturityNumber', 'obligorCreditScore', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue']))
max_cols = list(set(['gracePeriodNumber', 'interestPaidThroughDateR', 'mostRecentServicingTransferReceivedDateR', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber', 'repossessedProceedsAmount', 'currentDelinquencyStatus']))
all_vals = list(set(['subventedM', 'assetSubjectDemandIndicator', 'assetSubjectDemandStatusCodeM', 'repurchaserName', 'repurchaseReplacementReasonCodeM', 'modificationTypeCodeM', 'repossessedIndicator', 'zeroBalanceCodeM', 'accountStatus']))
sum_cols = list(set(['servicingFeePercentage', 'servicingFlatFeeAmount', 'otherServicerFeeRetainedByServicer', 'otherAssessedUncollectedServicerFeeAmount', 'totalActualAmountPaid', 'actualInterestCollectedAmount', 'actualPrincipalCollectedAmount', 'actualOtherCollectedAmount', 'servicerAdvancedAmount', 'currentDelinquencyStatus', 'repurchaseAmount', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber', 'repossessedProceedsAmount', 'repossessedProceedsAmount', 'gracePeriodNumber']))
capture_cols = list(set(['gracePeriodNumber', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue', 'reportingPeriodInterestRatePercentage', 'nextInterestRatePercentage', 'servicingFeePercentage', 'servicingFlatFeeAmount', 'otherServicerFeeRetainedByServicer', 'otherAssessedUncollectedServicerFeeAmount', 'scheduledInterestAmount', 'scheduledPrincipalAmount', 'otherPrincipalAdjustmentAmount', 'reportingPeriodActualEndBalanceAmount', 'reportingPeriodScheduledPaymentAmount', 'totalActualAmountPaid', 'actualInterestCollectedAmount', 'actualPrincipalCollectedAmount', 'actualOtherCollectedAmount', 'servicerAdvancedAmount', 'currentDelinquencyStatus', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber']))



In [ ]:
"""
1. Mark - take all rows prior to loan event
2. Mark - keep information from after event for final assessment
"""

In [32]:
def get_outcome(df):
    
    """
    Find remaining term
    """
    
    df = df.reset_index(drop = True)
    df = df.sort_values('reportingPeriodBeginningDateR', ascending = False)
    
    init_value = df['accountStatus'].iloc[0]
    if init_value in ['Prepaid or Matured', 'Charged-off', 'Repurchased or Replaced']:
        try:
            for idx, row in df.iterrows():
                row_val = row['accountStatus']
                next_val = df['accountStatus'].iloc[idx + 1]
                init_list = ['Unavailable', 'Charged-off', 'Prepaid or Matured']
                init_list.remove(row_val)
                if row_val == init_value and next_val in init_list:
                    return_val = df['remainingTermToMaturityNumber'].iloc[idx + 1]
                    date_val = df['reportingPeriodBeginningDateR'].iloc[idx]
                    return init_value, return_val, date_val
        except:
            return init_value, np.nan, np.nan
    else:
        return init_value, np.nan, np.nan
    

In [33]:
def reorganize(df, id_col, _id):
    
    """
    Static abs files
    """
    
    sub_df = df[df[id_col] == _id].reset_index(drop = True)
    _len = len(sub_df)
    
    for idx, row in sub_df.iterrows():
        val = row['accountStatus']
        if val in ['Charged-off', 'Prepaid or Matured', 'Repurchased or Replaced']:
            use_df = sub_df[idx:len(sub_df)].copy()
            break
    else:
        use_df = sub_df
    
    #act_status, timing, outcome_date = get_outcome(use_df)
    act_status, timing, outcome_date = get_outcome(use_df)
    
    reorder_dict = {}
    reorder_dict['ID'] = _id
    reorder_dict['records'] = _len
    reorder_dict['accountStatus'] = act_status
    reorder_dict['remainingTerm'] = timing
    reorder_dict['outcomeDate'] = outcome_date
    
    for col in iloc_cols:
        reorder_dict['{}Loc'.format(col)] = sub_df[col].iloc[0]
    
    for col in min_max_cols:
        _min = sub_df[col].min()
        _max = sub_df[col].max()
        reorder_dict['{}Min'.format(col)] = _min
        reorder_dict['{}Max'.format(col)] = _max
    
    for col in max_cols:
        reorder_dict['{}Max'.format(col)] = sub_df[col].max()
    
    for col in all_vals:
        vals = list(sub_df[col].unique())
        use_vals = ' | '.join(str(val) for val in vals)
        reorder_dict['{}Vals'.format(col)] = use_vals
    
    for col in sum_cols:
        reorder_dict['{}Sum'.format(col)] = sub_df[col].sum()
    
    df = pd.DataFrame(reorder_dict, index = [0])
    
    return df
        

### Application

In [34]:
all_ids = list(data['ID'].unique())
print_vals = list(range(0, len(all_ids), 100))

In [35]:
broke = []
master = pd.DataFrame()
count = 0
for _id in all_ids:
    
    df = reorganize(data, 'ID', _id)
    master = master.append(df).reset_index(drop = True)
    count = count + 1
    #print(_id)
#     try:
#         df = reorganize(data, 'ID', _id)
#         master = master.append(df).reset_index(drop = True)
#         count = count + 1
#     except:
#         print("can't parse {}".format(_id))
#         broke.append(_id)
#     if count in print_vals:
#         print(count)
#         print(count / len(all_ids))
#         print('---------------------------------------')
        

KeyboardInterrupt: 

In [ ]:
capture = ['gracePeriodNumber', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue', 'reportingPeriodInterestRatePercentage', 'nextInterestRatePercentage', 'servicingFeePercentage', 'servicingFlatFeeAmount', 'otherServicerFeeRetainedByServicer', 'otherAssessedUncollectedServicerFeeAmount', 'scheduledInterestAmount', 'scheduledPrincipalAmount', 'otherPrincipalAdjustmentAmount', 'reportingPeriodActualEndBalanceAmount', 'reportingPeriodScheduledPaymentAmount', 'totalActualAmountPaid', 'actualInterestCollectedAmount', 'actualPrincipalCollectedAmount', 'actualOtherCollectedAmount', 'servicerAdvancedAmount', 'currentDelinquencyStatus', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber']                



In [40]:
data

,assetTypeNumber,assetNumber,reportingPeriodBeginningDate,reportingPeriodEndingDate,originatorName,originationDate,originalLoanAmount,originalLoanTerm,loanMaturityDate,originalInterestRatePercentage,interestCalculationTypeCode,originalInterestRateTypeCode,originalInterestOnlyTermNumber,originalFirstPaymentDate,underwritingIndicator,gracePeriodNumber,paymentTypeCode,subvented,vehicleManufacturerName,vehicleModelName,vehicleNewUsedCode,vehicleModelYear,vehicleTypeCode,vehicleValueAmount,vehicleValueSourceCode,obligorCreditScoreType,obligorCreditScore,obligorIncomeVerificationLevelCode,obligorEmploymentVerificationCode,coObligorIndicator,paymentToIncomePercentage,obligorGeographicLocation,assetAddedIndicator,remainingTermToMaturityNumber,reportingPeriodModificationIndicator,servicingAdvanceMethodCode,reportingPeriodBeginningLoanBalanceAmount,nextReportingPeriodPaymentAmountDue,reportingPeriodInterestRatePercentage,nextInterestRatePercentage,servicingFeePercentage,servicingFlatFeeAmount,otherServicerFeeRetainedByServicer,otherAssessedUncollectedServicerFeeAmount,scheduledInterestAmount,scheduledPrincipalAmount,otherPrincipalAdjustmentAmount,reportingPeriodActualEndBalanceAmount,reportingPeriodScheduledPaymentAmount,totalActualAmountPaid,actualInterestCollectedAmount,actualPrincipalCollectedAmount,actualOtherCollectedAmount,servicerAdvancedAmount,interestPaidThroughDate,zeroBalanceEffectiveDate,zeroBalanceCode,currentDelinquencyStatus,primaryLoanServicerName,mostRecentServicingTransferReceivedDate,assetSubjectDemandIndicator,assetSubjectDemandStatusCode,repurchaseAmount,DemandResolutionDate,repurchaserName,repurchaseReplacementReasonCode,chargedoffPrincipalAmount,recoveredAmount,modificationTypeCode,paymentExtendedNumber,repossessedIndicator,repossessedProceedsAmount,dataset_name,ID,reportingPeriodBeginningDateR,reportingPeriodEndingDateR,originationDateR,loanMaturityDateR,originalFirstPaymentDateR,interestPaidThroughDateR,zeroBalanceEffectiveDateR,mostRecentServicingTransferReceivedDateR,DemandResolutionDateR,interestCalculationTypeCodeM,originalInterestRateTypeCodeM,paymentTypeCodeM,subventedM,vehicleNewUsedCodeM,vehicleTypeCodeM,vehicleValueSourceCodeM,obligorIncomeVerificationLevelCodeM,obligorEmploymentVerificationCodeM,servicingAdvanceMethodCodeM,zeroBalanceCodeM,assetSubjectDemandStatusCodeM,repurchaseReplacementReasonCodeM,modificationTypeCodeM,accountStatus
0,CIK number-Sequential asset number,"=""0001694010 - 000010""",11-01-2020,11-30-2020,GM FINANCIAL,03/2012,27746.63,74,12/2018,0.0890,1,1,0,05/2012,True,2,2,1,CHEVROLET,MALIBU,1,2012,1,24412.0,1,Credit Bureau Score,552,2,2,True,0.0688,CA,False,0,False,1,0.00,0.00,NaN,NaN,0.0225,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.00,386.18,75.03,311.15,0.0,0.0,-,07/2017,4,NaN,GM FINANCIAL,-,False,-,NaN,-,-,-,-311.15,0.0,-,0,False,0.0,2020.12.22_AmeriCredit Automobile Receivables Trust 2017-1 Data Tape.csv,0001694010 - 000010,2020-11-01,2020-11-30,2012-03,2018-12,2012-05,,2017-07,,,Simple,Fixed,Monthly,Yes - Rate Subvention,New,Car,Invoice Price,"Stated, not verified","Stated, not verified",No advancing,Charged-off,Other,Other,Other,Charged-off
1,CIK number-Sequential asset number,"=""0001694010 - 000088""",11-01-2020,11-30-2020,GM FINANCIAL,03/2012,22805.00,73,03/2018,0.1449,1,1,0,04/2012,True,1,2,0,DODGE,GRAND CARAVAN,2,2011,2,19225.0,98,Credit Bureau Score,561,2,2,False,0.0924,OH,False,0,False,1,0.00,0.00,NaN,NaN,0.0225,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.00,45.00,13.11,31.89,0.0,0.0,-,01/2018,4,NaN,GM FINANCIAL,-,False,-,NaN,-,-,-,-31.89,0.0,-,0,False,0.0,2020.12.22_AmeriCredit Automobile Receivables Trust 2017-1 Data Tape.csv,0001694010 - 000088,2020-11-01,2020-11-30,2012-03,2018-03,2012-04,,2018-01,,,Simple,Fixed,Monthly,No,Used,Truck,Other,"Stated, not verified","Stated, not verified",No advancing,Charged-off,Other,Other,Other,Charged-off
2,CIK number-Sequential asset number,"=""0001694010 - 000264""",11-01-2020,11-30-2020,GM FINANCIAL,03/2012,21166.38,73,06/2018,0.109

### Validation

In [36]:
master

,ID,records,accountStatus,remainingTerm,outcomeDate,originalFirstPaymentDateRLoc,primaryLoanServicerNameLoc,paymentToIncomePercentageLoc,originalLoanAmountLoc,loanMaturityDateRLoc,vehicleValueAmountLoc,reportingPeriodInterestRatePercentageLoc,reportingPeriodActualEndBalanceAmountLoc,vehicleModelYearLoc,obligorGeographicLocationLoc,obligorIncomeVerificationLevelCodeMLoc,servicingAdvanceMethodCodeMLoc,assetAddedIndicatorLoc,vehicleManufacturerNameLoc,vehicleNewUsedCodeMLoc,originalInterestRateTypeCodeMLoc,vehicleModelNameLoc,interestCalculationTypeCodeMLoc,assetSubjectDemandIndicatorLoc,scheduledInterestAmountLoc,otherPrincipalAdjustmentAmountLoc,originalInterestOnlyTermNumberLoc,originalInterestRatePercentageLoc,originatorNameLoc,reportingPeriodBeginningLoanBalanceAmountLoc,originationDateRLoc,zeroBalanceEffectiveDateRLoc,vehicleValueSourceCodeMLoc,obligorEmploymentVerificationCodeMLoc,reportingPeriodModificationIndicatorLoc,nextInterestRatePercentageLoc,originalLoanTermLoc,coObligorIndicatorLoc,paymentTypeCodeMLoc,vehicleTypeCodeMLoc,reportingPeriodScheduledPaymentAmountLoc,nextReportingPeriodPaymentAmountDueLoc,underwritingIndicatorLoc,obligorCreditScoreTypeLoc,reportingPeriodBeginningDateRMin,reportingPeriodBeginningDateRMax,remainingTermToMaturityNumberMin,remainingTermToMaturityNumberMax,obligorCreditScoreMin,obligorCreditScoreMax,reportingPeriodEndingDateRMin,reportingPeriodEndingDateRMax,gracePeriodNumberMax,currentDelinquencyStatusMax,chargedoffPrincipalAmountMax,mostRecentServicingTransferReceivedDateRMax,recoveredAmountMax,interestPaidThroughDateRMax,paymentExtendedNumberMax,repossessedProceedsAmountMax,repurchaseReplacementReasonCodeMVals,assetSubjectDemandIndicatorVals,modificationTypeCodeMVals,repossessedIndicatorVals,zeroBalanceCodeMVals,assetSubjectDemandStatusCodeMVals,subventedMVals,repurchaserNameVals,accountStatusVals,servicingFlatFeeAmountSum,otherServicerFeeRetainedByServicerSum,actualOtherCollectedAmountSum,gracePeriodNumberSum,servicingFeePercentageSum,currentDelinquencyStatusSum,chargedoffPrincipalAmountSum,totalActualAmountPaidSum,actualPrincipalCollectedAmountSum,otherAssessedUncollectedServicerFeeAmountSum,actualInterestCollectedAmountSum,recoveredAmountSum,repurchaseAmountSum,servicerAdvancedAmountSum,paymentExtendedNumberSum,repossessedProceedsAmountSum
0,0001694010 - 000010,27,Charged-off,18.0,2017-07-01,2012-05,GM FINANCIAL,0.0688,27746.63,2018-12,24412.0,NaN,0.00,2012,CA,"Stated, not verified",No advancing,False,CHEVROLET,New,Fixed,MALIBU,Simple,False,0.000000,0.0,0,0.0890,GM FINANCIAL,0.00,2012-03,2017-07,Invoice Price,"Stated, not verified",False,NaN,74,True,Monthly,Car,0.00,0.00,True,Credit Bureau Score,2016-12-01,2020-11-01,0,24,552,552,2016-12-31,2020-11-30,2,107.0,11927.53,,0.00,2017-04-21,0,0.0,Other,False,Other,False,Charged-off | Unavailable,Other,Yes - Rate Subvention,-,Charged-off | Unavailable,0.0,0.0,0.00,54,0.6075,364.0,9.935890e+03,6750.82,3138.01,1251.59,3612.81,0.00,0.0,0.0,0,0.0
1,0001694010 - 000088,40,Charged-off,3.0,2018-01-01,2012-04,GM FINANCIAL,0.0924,22805.00,2018-03,19225.0,NaN,0.00,2011,OH,"Stated, not verified",No advancing,False,DODGE,Used,Fixed,GRAND CARAVAN,Simple,False,0.000000,0.0,0,0.1449,GM FINANCIAL,0.00,2012-03,2018-01,Other,"Stated, not verified",False,NaN,73,False,Monthly,Truck,0.00,0.00,True,Credit Bureau Score,2016-12-01,2020-11-01,0,15,561,561,2016-12-31,2020-11-30,1,94.0,2136.83,,0.00,2017-12-15,0,0.0,Other,False,Other,False,Charged-off | Unavailable,Other,No,-,Charged-off | Unavailable,0.0,0.0,0.00,40,0.9000,192.0,1.348190e+03,5998.00,4665.34,0.00,1332.66,0.00,0.0,0.0,0,0.0
2,0001694010 - 000264,47,Prepaid or Matured,-2.0,2018-09-01,2012-04,GM FINANCIAL,0.0819,21166.38,2018-06,23472.0,0.10900000,0.28,2012,NY,"Stated, not verified",No advancing,False,CHEVROLET,New,Fixed,MALIBU,Simple,False,0.000000,0.0,0,0.1090,GM FINANCIAL,0.28,2012-03,,Invoice Price,"Stated, not verified",False,0.10900000,73,True,Monthly,Car,0.00,0.35,True,Credit Bureau Score,2016-12-0

In [37]:
master['accountStatus'].value_counts()

Unavailable           29
Charged-off           28
Prepaid or Matured    18
Name: accountStatus, dtype: int64

In [39]:
review_cols = ['ID', 'reportingPeriodBeginningDate', 'reportingPeriodEndingDate', 'originationDate', 'originalLoanTerm', 'originalLoanAmount', 'loanMaturityDate', 'remainingTermToMaturityNumber', 'reportingPeriodBeginningLoanBalanceAmount', 'reportingPeriodBeginningLoanBalanceAmount', 'totalActualAmountPaid', 'zeroBalanceEffectiveDate', 'chargedoffPrincipalAmount', 'repossessedIndicator', 'recoveredAmount', 'zeroBalanceCodeM', 'currentDelinquencyStatus', 'accountStatus', 'reportingPeriodInterestRatePercentage']             
sub_df = data[data['zeroBalanceCodeM'] == 'Prepaid or Matured']
ids = list(sub_df['ID'].unique())
_id = master['ID'].iloc[0]
sub = data[data['ID'] == _id].reset_index(drop = True)
sub[review_cols]

,ID,reportingPeriodBeginningDate,reportingPeriodEndingDate,originationDate,originalLoanTerm,originalLoanAmount,loanMaturityDate,remainingTermToMaturityNumber,reportingPeriodBeginningLoanBalanceAmount,reportingPeriodBeginningLoanBalanceAmount,totalActualAmountPaid,zeroBalanceEffectiveDate,chargedoffPrincipalAmount,repossessedIndicator,recoveredAmount,zeroBalanceCodeM,currentDelinquencyStatus,accountStatus,reportingPeriodInterestRatePercentage
0,0001694010 - 000010,11-01-2020,11-30-2020,03/2012,74,27746.63,12/2018,0,0.00,0.00,386.18,07/2017,-311.15,False,0.0,Charged-off,NaN,Charged-off,NaN
1,0001694010 - 000010,10-01-2020,10-31-2020,03/2012,74,27746.63,12/2018,0,0.00,0.00,270.00,07/2017,-210.22,False,0.0,Charged-off,NaN,Charged-off,NaN
2,0001694010 - 000010,09-01-2020,09-30-2020,03/2012,74,27746.63,12/2018,0,0.00,0.00,270.00,07/2017,-208.79,False,0.0,Charged-off,NaN,Charged-off,NaN
3,0001694010 - 000010,08-01-2020,08-31-2020,03/2012,74,27746.63,12/2018,0,0.00,0.00,270.00,07/2017,-209.60,False,0.0,Charged-off,NaN,Charged-off,NaN
4,0001694010 - 000010,07-01-2020,07-31-2020,03/2012,74,27746.63,12/2018,0,0.00,0.00,270.00,07/2017,-205.97,False,0.0,Charged-off,NaN,Charged-off,NaN
5,0001694010 - 000010,06-01-2020,06-30-2020,03/2012,74,27746.63,12/2018,0,0.00,0.00,270.00,07/2017,-169.88,False,0.0,Charged-off,NaN,Charged-off,NaN
6,0001694010 - 000010,05-01-2020,05-31-2020,03/2012,74,27746.63,12/2018,0,0.00,0.00,270.00,07/2017,-203.43,False,0.0,Charged-off,NaN,Charged-off,NaN
7,0001694010 - 000010,04-01-2020,04-30-2020,03/2012,74,27746.63,12/2018,0,0.00,0.00,270.00,07/2017,-202.04,False,0.0,Charged-off,NaN,Charged-off,NaN
8,0001694010 - 000010,03-01-2020,03-31-2020,03/2012,74,27746.63,12/2018,0,0.00,0.00,270.00,07/2017,-203.14,False,0.0,Charged-off,NaN,Charged-off,NaN
9,0001694010 - 000010,02-01-2020,02-29-2020,03/2012,74,27746.63,12/2018,0,0.00,0.00,270.00,07/2017,-57.61,False,0.0,Charged-off,NaN,Charged-off,NaN


In [42]:
sample = sub[review_cols]

In [45]:
sample[sample['accountStatus'].isin(['Charged-off'])].first_valid_index()


0

In [ ]:
for idx, row in sample.iterrows():
    

In [108]:
loc_cols = list(set(['originatorName', 'primaryLoanServicerName', 'originationDateR', 'originalLoanAmount', 'originalLoanTerm', 'loanMaturityDateR', 'originalInterestRatePercentage', 'interestCalculationTypeCodeM', 'originalInterestRateTypeCodeM', 'originalInterestOnlyTermNumber', 'originalFirstPaymentDateR', 'underwritingIndicator', 'paymentTypeCodeM', 'vehicleManufacturerName', 'vehicleModelName', 'vehicleNewUsedCodeM', 'vehicleModelYear', 'vehicleTypeCodeM', 'vehicleValueAmount', 'vehicleValueSourceCodeM', 'obligorCreditScoreType', 'obligorIncomeVerificationLevelCodeM', 'obligorEmploymentVerificationCodeM', 'coObligorIndicator', 'paymentToIncomePercentage', 'obligorGeographicLocation', 'assetAddedIndicator', 'reportingPeriodModificationIndicator', 'servicingAdvanceMethodCodeM', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue', 'reportingPeriodInterestRatePercentage', 'nextInterestRatePercentage', 'scheduledInterestAmount', 'otherPrincipalAdjustmentAmount', 'reportingPeriodActualEndBalanceAmount', 'reportingPeriodScheduledPaymentAmount', 'assetSubjectDemandIndicator', 'zeroBalanceEffectiveDateR']))         
min_max_cols = list(set(['reportingPeriodBeginningDateR', 'reportingPeriodEndingDateR', 'remainingTermToMaturityNumber', 'obligorCreditScore', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue']))
max_cols = list(set(['gracePeriodNumber', 'interestPaidThroughDateR', 'mostRecentServicingTransferReceivedDateR', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber', 'repossessedProceedsAmount', 'currentDelinquencyStatus']))
all_vals = list(set(['subventedM', 'assetSubjectDemandIndicator', 'assetSubjectDemandStatusCodeM', 'repurchaserName', 'repurchaseReplacementReasonCodeM', 'modificationTypeCodeM', 'repossessedIndicator', 'zeroBalanceCodeM', 'accountStatus']))
sum_cols = list(set(['servicingFeePercentage', 'servicingFlatFeeAmount', 'otherServicerFeeRetainedByServicer', 'otherAssessedUncollectedServicerFeeAmount', 'totalActualAmountPaid', 'actualInterestCollectedAmount', 'actualPrincipalCollectedAmount', 'actualOtherCollectedAmount', 'servicerAdvancedAmount', 'currentDelinquencyStatus', 'repurchaseAmount', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber', 'repossessedProceedsAmount', 'repossessedProceedsAmount', 'gracePeriodNumber']))
capture_cols = list(set(['gracePeriodNumber', 'reportingPeriodBeginningLoanBalanceAmount', 'nextReportingPeriodPaymentAmountDue', 'reportingPeriodInterestRatePercentage', 'nextInterestRatePercentage', 'servicingFeePercentage', 'servicingFlatFeeAmount', 'otherServicerFeeRetainedByServicer', 'otherAssessedUncollectedServicerFeeAmount', 'scheduledInterestAmount', 'scheduledPrincipalAmount', 'otherPrincipalAdjustmentAmount', 'reportingPeriodActualEndBalanceAmount', 'reportingPeriodScheduledPaymentAmount', 'totalActualAmountPaid', 'actualInterestCollectedAmount', 'actualPrincipalCollectedAmount', 'actualOtherCollectedAmount', 'servicerAdvancedAmount', 'currentDelinquencyStatus', 'chargedoffPrincipalAmount', 'recoveredAmount', 'paymentExtendedNumber']))
event_cols = list(set(['accountStatus', 'zeroBalanceCodeM', 'chargedoffPrincipalAmount', 'reportingPeriodBeginningDateR', 'reportingPeriodEndingDateR']))


In [109]:
id_col = 'ID'
_id = '0001694010 - 000010'
column = 'accountStatus'

In [121]:
df = data[data[id_col] == _id].reset_index(drop = True)
df = df.sort_values('reportingPeriodBeginningDateR', ascending = False)

# dict
account_dict = {}
account_dict['id'] = _id
account_dict['records'] = len(df)

# grab current info
for col in loc_cols:
    account_dict['{}LocCurrent'.format(col)] = df[col].iloc[0]
for col in min_max_cols:
    _min = df[col].min()
    _max = df[col].max()
    account_dict['{}MinCurrent'.format(col)] = _min
    account_dict['{}MaxCurrent'.format(col)] = _max
for col in max_cols:
    account_dict['{}MaxCurrent'.format(col)] = df[col].max()
for col in all_vals:
    vals = list(df[col].unique())
    use_vals = ' | '.join(str(val) for val in vals)
    account_dict['{}ValsCurrent'.format(col)] = use_vals
for col in sum_cols:
    account_dict['{}SumCurrent'.format(col)] = df[col].sum()

# event
init_vals = list(df[column].unique())
inter = list(set(values).intersection(init_vals))
if len(inter) > 0:
    account_dict['eventOccured'] = 1
    n = df[column].where(df[column].isin(values)).last_valid_index()
    single = df.loc[[n]]
    for col in event_cols:
        account_dict['{}Event'.format(col)] = single[col].iloc[0]

    # prior to event
    init = n+1
    sub = df[init:len(df)]
    sub.reset_index(drop = True, inplace = True)
    
    for col in loc_cols:
        account_dict['{}LocPrior'.format(col)] = sub[col].iloc[0]
    for col in min_max_cols:
        _min = sub[col].min()
        _max = sub[col].max()
        account_dict['{}MinPrior'.format(col)] = _min
        account_dict['{}MaxPrior'.format(col)] = _max
    for col in max_cols:
        account_dict['{}MaxPrior'.format(col)] = sub[col].max()
    for col in all_vals:
        vals = list(sub[col].unique())
        use_vals = ' | '.join(str(val) for val in vals)
        account_dict['{}ValsPrior'.format(col)] = use_vals
    for col in sum_cols:
        account_dict['{}SumPrior'.format(col)] = sub[col].sum()
    
else:
    account_dict['eventOccured'] = 0

    for col in loc_cols:
        account_dict['{}LocPrior'.format(col)] = np.nan
    for col in min_max_cols:
        _min = df[col].min()
        _max = df[col].max()
        account_dict['{}MinPrior'.format(col)] = np.nan
        account_dict['{}MaxPrior'.format(col)] = np.nan
    for col in max_cols:
        account_dict['{}MaxPrior'.format(col)] = np.nan
    for col in all_vals:
        account_dict['{}ValsPrior'.format(col)] = np.nan
    for col in sum_cols:
        account_dict['{}SumPrior'.format(col)] = np.nan
        

In [122]:
account_dict

{'id': '0001694010 - 000010',
 'records': 27,
 'originalFirstPaymentDateRLocCurrent': '2012-05',
 'primaryLoanServicerNameLocCurrent': 'GM FINANCIAL',
 'paymentToIncomePercentageLocCurrent': 0.0688,
 'originalLoanAmountLocCurrent': 27746.63,
 'loanMaturityDateRLocCurrent': '2018-12',
 'vehicleValueAmountLocCurrent': 24412.0,
 'reportingPeriodInterestRatePercentageLocCurrent': nan,
 'reportingPeriodActualEndBalanceAmountLocCurrent': 0.0,
 'vehicleModelYearLocCurrent': 2012,
 'obligorGeographicLocationLocCurrent': 'CA',
 'obligorIncomeVerificationLevelCodeMLocCurrent': 'Stated, not verified',
 'servicingAdvanceMethodCodeMLocCurrent': 'No advancing',
 'assetAddedIndicatorLocCurrent': False,
 'vehicleManufacturerNameLocCurrent': 'CHEVROLET',
 'vehicleNewUsedCodeMLocCurrent': 'New',
 'originalInterestRateTypeCodeMLocCurrent': 'Fixed',
 'vehicleModelNameLocCurrent': 'MALIBU',
 'interestCalculationTypeCodeMLocCurrent': 'Simple',
 'assetSubjectDemandIndicatorLocCurrent': False,
 'scheduledInte

In [123]:
sample = pd.DataFrame(account_dict, index = [0])

In [124]:
sample

,id,records,originalFirstPaymentDateRLocCurrent,primaryLoanServicerNameLocCurrent,paymentToIncomePercentageLocCurrent,originalLoanAmountLocCurrent,loanMaturityDateRLocCurrent,vehicleValueAmountLocCurrent,reportingPeriodInterestRatePercentageLocCurrent,reportingPeriodActualEndBalanceAmountLocCurrent,vehicleModelYearLocCurrent,obligorGeographicLocationLocCurrent,obligorIncomeVerificationLevelCodeMLocCurrent,servicingAdvanceMethodCodeMLocCurrent,assetAddedIndicatorLocCurrent,vehicleManufacturerNameLocCurrent,vehicleNewUsedCodeMLocCurrent,originalInterestRateTypeCodeMLocCurrent,vehicleModelNameLocCurrent,interestCalculationTypeCodeMLocCurrent,assetSubjectDemandIndicatorLocCurrent,scheduledInterestAmountLocCurrent,otherPrincipalAdjustmentAmountLocCurrent,originalInterestOnlyTermNumberLocCurrent,originalInterestRatePercentageLocCurrent,originatorNameLocCurrent,reportingPeriodBeginningLoanBalanceAmountLocCurrent,originationDateRLocCurrent,zeroBalanceEffectiveDateRLocCurrent,vehicleValueSourceCodeMLocCurrent,obligorEmploymentVerificationCodeMLocCurrent,reportingPeriodModificationIndicatorLocCurrent,nextInterestRatePercentageLocCurrent,originalLoanTermLocCurrent,coObligorIndicatorLocCurrent,paymentTypeCodeMLocCurrent,vehicleTypeCodeMLocCurrent,reportingPeriodScheduledPaymentAmountLocCurrent,nextReportingPeriodPaymentAmountDueLocCurrent,underwritingIndicatorLocCurrent,obligorCreditScoreTypeLocCurrent,reportingPeriodBeginningLoanBalanceAmountMinCurrent,reportingPeriodBeginningLoanBalanceAmountMaxCurrent,obligorCreditScoreMinCurrent,obligorCreditScoreMaxCurrent,reportingPeriodBeginningDateRMinCurrent,reportingPeriodBeginningDateRMaxCurrent,reportingPeriodEndingDateRMinCurrent,reportingPeriodEndingDateRMaxCurrent,nextReportingPeriodPaymentAmountDueMinCurrent,nextReportingPeriodPaymentAmountDueMaxCurrent,remainingTermToMaturityNumberMinCurrent,remainingTermToMaturityNumberMaxCurrent,gracePeriodNumberMaxCurrent,currentDelinquencyStatusMaxCurrent,chargedoffPrincipalAmountMaxCurrent,mostRecentServicingTransferReceivedDateRMaxCurrent,recoveredAmountMaxCurrent,interestPaidThroughDateRMaxCurrent,paymentExtendedNumberMaxCurrent,repossessedProceedsAmountMaxCurrent,repurchaseReplacementReasonCodeMValsCurrent,assetSubjectDemandIndicatorValsCurrent,modificationTypeCodeMValsCurrent,repossessedIndicatorValsCurrent,zeroBalanceCodeMValsCurrent,assetSubjectDemandStatusCodeMValsCurrent,subventedMValsCurrent,repurchaserNameValsCurrent,accountStatusValsCurrent,servicingFlatFeeAmountSumCurrent,otherServicerFeeRetainedByServicerSumCurrent,actualOtherCollectedAmountSumCurrent,gracePeriodNumberSumCurrent,servicingFeePercentageSumCurrent,currentDelinquencyStatusSumCurrent,chargedoffPrincipalAmountSumCurrent,totalActualAmountPaidSumCurrent,actualPrincipalCollectedAmountSumCurrent,otherAssessedUncollectedServicerFeeAmountSumCurrent,actualInterestCollectedAmountSumCurrent,recoveredAmountSumCurrent,repurchaseAmountSumCurrent,servicerAdvancedAmountSumCurrent,paymentExtendedNumberSumCurrent,repossessedProceedsAmountSumCurrent,eventOccured,zeroBalanceCodeMEvent,chargedoffPrincipalAmountEvent,reportingPeriodBeginningDateREvent,reportingPeriodEndingDateREvent,accountStatusEvent,originalFirstPaymentDateRLocPrior,primaryLoanServicerNameLocPrior,paymentToIncomePercentageLocPrior,originalLoanAmountLocPrior,loanMaturityDateRLocPrior,vehicleValueAmountLocPrior,reportingPeriodInterestRatePercentageLocPrior,reportingPeriodActualEndBalanceAmountLocPrior,vehicleModelYearLocPrior,obligorGeographicLocationLocPrior,obligorIncomeVerificationLevelCodeMLocPrior,servicingAdvanceMethodCodeMLocPrior,assetAddedIndicatorLocPrior,vehicleManufacturerNameLocPrior,vehicleNewUsedCodeMLocPrior,originalInterestRateTypeCodeMLocPrior,vehicleModelNameLocPrior,interestCalculationTypeCodeMLocPrior,assetSubjectDemandIndicatorLocPrior,scheduledInterestAmountLocPrior,otherPrincipalAdjustmentAmountLocPrior,originalInterestOnlyTermNumberLocPrior,originalInterestRatePercentageLocPrior,originatorNameLocPrior,rep

In [86]:
def get_prior(data, id_col, _id, column, values):
    
    """
    Get history
    """
    
    df = data[data[id_col] == _id].reset_index(drop = True)
    df = df.sort_values('reportingPeriodBeginningDateR', ascending = False)
    
    # dict
    account_dict = {}
    account_dict['id'] = _id
    
    for col in loc_cols:
        account_dict['{}LocCurrent'.format(col)] = df[col].iloc[0]
    
    for col in min_max_cols:
        _min = sub_df[col].min()
        _max = sub_df[col].max()
        reorder_dict['{}MinCurrent'.format(col)] = _min
        reorder_dict['{}MaxCurreent'.format(col)] = _max
    
    for col in max_cols:
        reorder_dict['{}MacCurrent'.format(col)] = sub_df[col].max()
    
    for col in all_vals:
        vals = list(sub_df[col].unique())
        use_vals = ' | '.join(str(val) for val in vals)
        reorder_dict['{}ValsCurrent'.format(col)] = use_vals
    
    for col in sum_cols:
        reorder_dict['{}SumCurrent'.format(col)] = sub_df[col].sum()
        
    init_vals = list(df[column].unique())
    inter = list(set(values).intersection(init_vals))
        
    if len(inter) > 0:
        n = df[column].where(df[column].isin(values)).last_valid_index()
        single = df.loc[[n]]
        
        for col in event_cols:
            account_dict['{}Event'.format(col)] = single[col].iloc[0]
    
        init_sub = df[n:len(df)]
        init = n+1
        sub = df[init:len(df)]
        sub.reset_index(drop = True, inplace = True)
        _class = 'E'
    else:
        sub = df
        _class = 'NE'
    
    return sub, _class


In [87]:
col = 'accountStatus'
values = ['Charged-off', 'Prepaid or Matured', 'Repurchased or Replaced']


In [88]:
df, val = get_prior(sample, col, values)

In [47]:
sample[20:len(sample)]

,ID,reportingPeriodBeginningDate,reportingPeriodEndingDate,originationDate,originalLoanTerm,originalLoanAmount,loanMaturityDate,remainingTermToMaturityNumber,reportingPeriodBeginningLoanBalanceAmount,reportingPeriodBeginningLoanBalanceAmount,totalActualAmountPaid,zeroBalanceEffectiveDate,chargedoffPrincipalAmount,repossessedIndicator,recoveredAmount,zeroBalanceCodeM,currentDelinquencyStatus,accountStatus,reportingPeriodInterestRatePercentage
20,0001694010 - 000010,07-01-2017,07-31-2017,03/2012,74,27746.63,12/2018,0,11927.53,11927.53,185.34,07/2017,11927.53,False,0.0,Charged-off,NaN,Charged-off,0.08900000
21,0001694010 - 000010,06-01-2017,06-30-2017,03/2012,74,27746.63,12/2018,18,11927.53,11927.53,0.00,-,0.00,False,0.0,Unavailable,107.0,Unavailable,0.08900000
22,0001694010 - 000010,05-01-2017,05-31-2017,03/2012,74,27746.63,12/2018,19,11927.53,11927.53,0.00,-,0.00,False,0.0,Unavailable,77.0,Unavailable,0.08900000
23,0001694010 - 000010,04-01-2017,04-30-2017,03/2012,74,27746.63,12/2018,20,12094.95,12094.95,250.00,-,0.00,False,0.0,Unavailable,46.0,Unavailable,0.08900000
24,0001694010 - 000010,03-01-2017,03-31-2017,03/2012,74,27746.63,12/2018,21,12469.28,12469.28,501.25,-,0.00,False,0.0,Unavailable,44.0,Unavailable,0.089
25,0001694010 - 000010,02-01-2017,02-28-2017,03/2012,74,27746.63,12/2018,22,12935.20,12935.20,500.62,-,0.00,False,0.0,Unavailable,44.0,Unavailable,0.089
26,0001694010 - 000010,12-01-2016,12-31-2016,03/2012,74,27746.63,12/2018,24,13461.31,13461.31,250.00,-,0.00,False,0.0,Unavailable,46.0,Unavailable,0.089


In [ ]:
accountStatus, zeroBalanceCodeM, chargedoffPrincipalAmount, chargedoffPrincipalAmountR, reportingPeriodEndingDateR

### Export

In [33]:
e_folder = 'data/static/'
e_file = '{} static.csv'.format(term)
e_path = e_folder + e_file
e_path

'data/static/AmeriCredit Automobile Receivables Trust 2017-1 Data Tape static.csv'

In [34]:
master.to_csv(e_path, index = False)

In [35]:
print('continue...')

continue...


### End